In [1]:
from utils import extract_all_features_for_count_threshold, extract_features_for_dist_threshold, get_custom_matrix_features
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
from utils import get_matrix_features
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')

In [3]:
import pandas as pd
feature_list = ['']
threshold_mode = 'count'
num_nn = 4
# colony 0

_19_f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]


all_candidate_features = pd.read_csv(os.path.join(path_to_data, 'all_candidate_features_19_{}_{}_nn_{}_thresh.csv'.format(threshold_mode, num_nn, 10)))
all_gt = pd.concat([colony0_lineage_gt,colonies_gt]).reset_index()
# all_candidate_features.to_csv(os.path.join(path_to_data, 'all_candidate_features_19_{}_{}_nn_{}_thresh.csv'.format(threshold_mode, num_nn, 10)), index=False)
all_matrix_features = get_custom_matrix_features(all_candidate_features, all_gt, _19_f_list)
all_matrix_features

,index,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
4,4,1,5,18,0,"[[2.23606797749979, 2.8284271247461903, 2.0, 0...","[1, 3, 2, 4]",0
5,5,3,6,18,0,"[[1.0, 3.0, 1.0, 0.7071067811865476, 3.1110787...","[3, 4, 2, 1]",0
6,6,2,7,20,0,"[[1.0, 3.0, 1.0, 0.7512255508302275, 0.9423480...","[2, 3, 4, 6]",0
7,7,4,8,22,0,"[[1.0, 2.0, 1.0, 0.5, 2.107542585550348, 2.122...","[4, 7, 6, 2]",0
8,8,1,9,32,0,"[[2.0, 2.0, 1.0, 0.4330127018922193, 1.7589301...","[1, 5, 3, 2]",0
...,...,...,...,...,...,...,...,...
867,499,22,34,164,5,"[[2.0, 3.0, 2.0, 0.4330127018922193, 1.1800917...","[22, 25, 8, 11]",0
868,500,13,35,166,5,"[[2.0, 3.0, 2.0, 0.4611141570664294, 0.8067932...","[13, 16, 10, 4]",0
869,501,18,36,169,5,"[[2.23606797749979, 4.123105625617661, 2.23606...","[18, 9, 2, 32]",0
870,502,3,37,175,5,"[[2.0, 2.23606797749979, 2.0, 0.10222043276741...","[18, 26, 3, 27]",2


## Apply min-max normalization on all data


In [6]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

_19_f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

# Extract the subset of features to be normalized as a separate DataFrame
X = all_candidate_features[_19_f_list]

# Apply min-max normalization to the subset of features
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

# Replace the original features with the normalized features in the dataframe
all_candidate_features_normalized = all_candidate_features.copy()
all_candidate_features_normalized[_19_f_list] = X_norm

all_matrix_features_normalized = get_custom_matrix_features(all_candidate_features_normalized, all_gt, _19_f_list)
all_matrix_features_normalized

,index,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
4,4,1,5,18,0,"[[0.0056879177484625105, 0.007189664377899039,...","[1, 3, 2, 4]",0
5,5,3,6,18,0,"[[0.0025437141471979447, 0.007625790654101655,...","[3, 4, 2, 1]",0
6,6,2,7,20,0,"[[0.0025437141471979447, 0.007625790654101655,...","[2, 3, 4, 6]",0
7,7,4,8,22,0,"[[0.0025437141471979447, 0.00508386043606777, ...","[4, 7, 6, 2]",0
8,8,1,9,32,0,"[[0.005087428294395889, 0.00508386043606777, 0...","[1, 5, 3, 2]",0
...,...,...,...,...,...,...,...,...
867,499,22,34,164,5,"[[0.005087428294395889, 0.007625790654101655, ...","[22, 25, 8, 11]",0
868,500,13,35,166,5,"[[0.005087428294395889, 0.007625790654101655, ...","[13, 16, 10, 4]",0
869,501,18,36,169,5,"[[0.0056879177484625105, 0.010480646781903039,...","[18, 9, 2, 32]",0
870,502,3,37,175,5,"[[0.005087428294395889, 0.005683928761584629, ...","[18, 26, 3, 27]",2


In [7]:
all_candidate_features_normalized

,index,bud_id,candid_id,time_id,dist_0,dist_max,dist_min,dist_std,poly_fit_budcm_candidcm,poly_fit_budcm_budpt,...,position_bud_last,position_bud_first,orientation_bud_std,orientation_bud_max,orientation_bud_min,orientation_bud_last,orientation_bud_first,orientation_bud_last_minus_first,plyfit_orientation_bud,colony
0,0,1,1,0,0.000000,0.000000,0.000000,0.000000,0.073362,0.071509,...,0.119632,0.573714,0.703661,0.913176,0.122687,0.119740,0.573706,0.267876,0.192929,0
1,0,1,3,0,0.003597,0.007190,0.003615,0.003659,0.083730,0.080355,...,0.390902,0.335416,0.466070,0.530134,0.011596,0.011273,0.042056,0.483463,0.344369,0
2,0,1,2,0,0.106866,0.106791,0.099809,0.006598,0.081782,0.082196,...,0.620064,0.823480,0.236400,0.432822,0.144278,0.373099,0.441327,0.464459,0.403358,0
3,0,1,4,0,0.138790,0.138902,0.137636,0.001930,0.092440,0.095096,...,0.743815,0.559642,0.245244,0.381482,0.101471,0.111525,0.198176,0.455019,0.348444,0
4,0,2,2,0,0.000000,0.000000,0.000000,0.000000,0.073362,0.071161,...,0.788922,0.281963,0.456605,0.785605,0.288792,0.788822,0.281852,0.757603,0.519523,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321,1899,37,27,175,0.043243,0.057965,0.043449,0.012929,0.104544,0.103660,...,0.018067,0.454703,0.067581,0.428907,0.356060,0.367833,0.392445,0.486680,0.386434,5
3322,1900,38,14,178,0.003597,0.007190,0.000000,0.006506,0.086753,0.084061,...,0.758748,0.749670,0.813786,0.933794,0.000000,0.852345,0.934979,0.457216,0.362123,5
3323,1901,38,23,178,0.007195,0.007626,0.000000,0.007303,0.086081,0.084252,...,0.594385,0.632859,0.294459,0.357327,0.000000,0.286631,0.366935,0.458288,0.363152,5
3324,1902,38,9,178,0.007631,0.007626,0.000000,0.006679,0.078768,0.076036,...,0.516339,0.592513,0.604685,0.725806,0.000000,0.729920,0.540636,0.595732,0.495094,5


## Train NN on normalized features


In [12]:
from nn_model import BudDataset, LineageNN, train_nn, test_nn, cv_nn


In [13]:
# for the full list of features normalized
f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features_normalized, all_gt, f_list,)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all', 'Group': 'normalized'}
models_n, accuracies_n = cv_nn(matrix_feat_custom, config=config)
# for the full list of features not normalized
f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list,)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all','Group': 'normalized'}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

print('accuracy for normalized features: ', np.mean(accuracies_n))
print('accuracy for not normalized features: ', np.mean(accuracies))

config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'dist_min', 'dist_std', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'Group': 'normalized', 'cv_number': 1}
yes. I am reloaded


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: f-labbaf97. Use `wandb login --relogin` to force relogin


early stopping at  28 LR:  0.007128999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'dist_min', 'dist_std', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'Group': 'normalized', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▅▅▅▆▇▇▇▇███████████████████
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
eval_accuracy,▁▅▄▄▆▇▇▃▇██▇▇▇█▅▇███▇██▆██▇▇
lr,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
patience,▁▁▂▂▁▁▂▂▃▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▆▇▇▇▇▇▇███████████████████
best_accuracy,0.8494
epoch,27
eval_accuracy,0.83133
lr,0.00723
patience,10


early stopping at  25 LR:  0.007425999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'dist_min', 'dist_std', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'Group': 'normalized', 'cv_number': 3}
yes. I am reloaded


...

best_accuracy,▁▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██████████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
eval_accuracy,▁▃▄▄▃▅▅▅▄▆▂▅▆▇▄▅▆▆▇▆█▆▇█▆▆▆▆▆▅▇▇▇▇
lr,███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▁▁▁▂▁▂▂▃▁▂▂▃▁▂▂▃▄▅▅▁▂▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████
best_accuracy,0.87879
epoch,33
eval_accuracy,0.85455
lr,0.00663
patience,10


early stopping at  19 LR:  0.008019999999999998
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'dist_min', 'dist_std', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'Group': 'normalized', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▃▅▇▇▇▇▇███████████
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
eval_accuracy,▁▃▅▇▇▇▆▆█▄▇█▇▆▄▆▅▇▇
lr,██▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
patience,▁▁▁▁▂▂▃▄▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▅▆▆▇▇▇▇▇█▇█████
best_accuracy,0.82424
epoch,18
eval_accuracy,0.80606
lr,0.00812
patience,10


early stopping at  29 LR:  0.007029999999999997
accuracy:  0.8450383351588171 +/- 0.02047988541008536
accuracy for normalized features:  0.8377583059510771
accuracy for not normalized features:  0.8450383351588171


wandb: Network error (ConnectionError), entering retry loop.
